In [1]:
library("dplyr")
library("tibble")
library("purrr")
library("readr")
library("tidyr")



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [4]:

## Define function
allele.qc  = function(a1,a2,ref1,ref2) {
        a1 = toupper(a1)
        a2 = toupper(a2)
        ref1 = toupper(ref1)
        ref2 = toupper(ref2)

    ref = ref1
    flip = ref
    flip[ref == "A"] = "T"
    flip[ref == "T"] = "A"
    flip[ref == "G"] = "C"
    flip[ref == "C"] = "G"
    flip1 = flip

    ref = ref2
    flip = ref
    flip[ref == "A"] = "T"
    flip[ref == "T"] = "A"
    flip[ref == "G"] = "C"
    flip[ref == "C"] = "G"
    flip2 = flip;

    snp = list()
    snp[["keep"]] = !((a1=="A" & a2=="T") | (a1=="T" & a2=="A") | (a1=="C" & a2=="G") | (a1=="G" & a2=="C"))
    snp[["keep"]][ a1 != "A" & a1 != "T" & a1 != "G" & a1 != "C" ] = F
    snp[["keep"]][ a2 != "A" & a2 != "T" & a2 != "G" & a2 != "C" ] = F
    snp[["flip"]] = (a1 == ref2 & a2 == ref1) | (a1 == flip2 & a2 == flip1)

    return(snp)
}

 allele.process = function(sumstat,name,n,stat){
    qc = allele.qc(sumstat$V2.y,sumstat$V3.y,sumstat$V2.x,sumstat$V3.x)
    # Flip Z-scores for mismatching alleles
    qc$flip = qc$flip%>%which%>%na.omit
    qc$keep = qc$keep%>%which%>%na.omit
    stat = which(colnames(sumstat) %in% stat)
    sumstat[ qc$flip,stat ] = -1 * sumstat[ qc$flip,stat ]
    sumstat$V2.y[ qc$flip ] = sumstat$V2.x[ qc$flip ]
    sumstat$V3.y[ qc$flip ] = sumstat$V3.x[ qc$flip ]
    # Remove strand ambiguous SNPs
        sumstat = sumstat[qc$keep,]
    # Merge the tags
        sumstat = sumstat%>%mutate(rowname.y = pmap_chr( list(V1,V2.y,V3.y),function(a,b,c) paste(c(a,b,c),collapse = "_")),
                                    rowname.x = coalesce(rowname.x,NA,rowname.y),
                                    V2.x = coalesce(V2.x,NA,V2.y),
                                    V3.x = coalesce(V3.x,NA,V3.y),
                                    )%>%select(-rowname.y,-V2.y,-V3.y)
    # Clean name
    colnames(sumstat)[c(1,2,4,5,n+4)] = c("rowname","bhat","V2","V3",name[n])
    return(sumstat)
    }


  ## Start


Theme = read_delim('/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis/Theme_list.txt', delim = "\t")
Theme = Theme%>%mutate(dir = map_chr(`#Theme`,~paste(c(`.x`,"/ENSG00000015475.rds"),collapse = "")),
                                      i = 1:nrow(Theme),
                                      bhat = map(dir, ~readRDS(.x)$bhat%>%as.data.frame%>%rownames_to_column%>%rename(bhat = 2)%>%mutate(snps = map(rowname,~read.table(text = .x, sep = "_",colClass = "character")%>%as_tibble ))%>%unnest(snps)),
                                      sbhat = map(dir, ~readRDS(.x)$sbhat%>%as.data.frame%>%rownames_to_column%>%rename(sbhat = 2)%>%mutate(snps = map(rowname,~read.table(text = .x, sep = "_",colClass = "character")%>%as_tibble ))%>%unnest(snps))
                                      )



── Column specification ────────────────────────────────────────────────────────
cols(
  `#Theme` = col_character()
)




In [7]:
genos_join_bhat = allele.process(genos_join_bhat,name,2,"bhat.y")


Warning message:
“Unknown or uninitialised column: `V2.y`.”
Warning message:
“Unknown or uninitialised column: `V3.y`.”
Warning message:
“Unknown or uninitialised column: `V2.x`.”
Warning message:
“Unknown or uninitialised column: `V3.x`.”
Warning message:
“Unknown or uninitialised column: `V2.x`.”
Warning message:
“Unknown or uninitialised column: `V2.y`.”
Warning message:
“Unknown or uninitialised column: `V3.x`.”
Warning message:
“Unknown or uninitialised column: `V3.y`.”


ERROR: Error: Can't transform a data frame with duplicate names.


In [8]:
genos_join_bhat

rowname,NA,V1,V2,V3,NA
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
22:17235588_C_T,0.07460630,22:17235588,C,T,0.0354
22:17235607_G_A,-0.00270508,22:17235607,G,A,0.0144
22:17236001_C_T,-0.01795600,22:17236001,C,T,0.0390
22:17237001_A_G,0.05690610,22:17237001,A,G,0.0255
22:17237039_T_C,0.10258700,22:17237039,T,C,0.0295
22:17239061_A_G,-0.00801436,22:17239061,A,G,0.0190
22:17239070_T_G,0.05708380,22:17239070,T,G,0.0245
22:17239082_A_G,0.05579030,22:17239082,A,G,0.0287
22:17239102_C_T,0.01849010,22:17239102,C,T,0.0117


In [10]:
n = nrow(Theme)
name = Theme%>%mutate(name = map(`#Theme`, ~read.table(text = .x,sep = "/")),
                            name = map_chr(name, ~.x[,ncol(.x)-2]%>%as.character) )%>%pull(name)


# Join first two conditions
genos_join_bhat = full_join((Theme%>%pull(bhat))[[1]],(Theme%>%pull(bhat))[[2]],by = "V1")

genos_join_bhat = allele.process(genos_join_bhat,name,2,"bhat.y")



genos_join_sbhat = full_join((Theme%>%pull(sbhat))[[1]],(Theme%>%pull(sbhat))[[2]],by = "V1")

genos_join_sbhat  = allele.process(genos_join_sbhat,name,2,"sbhat.y")


# If there are more conditions, join the rest
if(n > 2){
    for(j in 3:n){
        genos_join_bhat = full_join(genos_join_bhat,(Theme%>%pull(bhat))[[j]],by = "V1")
        genos_join_bhat = allele.process(genos_join_bhat,name,j,"bhat.y")
        genos_join_sbhat = full_join(genos_join_sbhat,(Theme%>%pull(sbhat))[[j]],by = "V1")
        genos_join_sbhat  = allele.process(genos_join_sbhat,name,j,"sbhat.y")
    }
}
colnames(genos_join_bhat)[2] = name[1]
colnames(genos_join_sbhat)[2] = name[1]


In [ ]:
Theme%>%mutate(name = map(`#Theme`, ~read.table(text = .x,sep = "/")),
                            name = map_chr(name, ~.x[,ncol(.x)-2]%>%as.character) )

In [14]:
Theme

In [ ]:

snps = genos_join_bhat$rowname
genos_join_bhat = genos_join_bhat%>%select(-rowname,-V1,-V2,-V3)%>%as.matrix
genos_join_sbhat = genos_join_sbhat%>%select(-rowname,-V1,-V2,-V3)%>%as.matrix%>%abs

# save the rds file
saveRDS(file = "/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis/sumstat/RDS/ENSG00000015475.rds", list(bhat=genos_join_bhat, sbhat=genos_join_sbhat,Z = genos_join_bhat/genos_join_sbhat ,snps = snps))

In [16]:
library("dplyr")
library("tibble")
library("purrr")
library("readr")
library("tidyr")

## Define function
allele.qc  = function(a1,a2,ref1,ref2) {
        a1 = toupper(a1)
        a2 = toupper(a2)
        ref1 = toupper(ref1)
        ref2 = toupper(ref2)

    ref = ref1
    flip = ref
    flip[ref == "A"] = "T"
    flip[ref == "T"] = "A"
    flip[ref == "G"] = "C"
    flip[ref == "C"] = "G"
    flip1 = flip

    ref = ref2
    flip = ref
    flip[ref == "A"] = "T"
    flip[ref == "T"] = "A"
    flip[ref == "G"] = "C"
    flip[ref == "C"] = "G"
    flip2 = flip;

    snp = list()
    snp[["keep"]] = !((a1=="A" & a2=="T") | (a1=="T" & a2=="A") | (a1=="C" & a2=="G") | (a1=="G" & a2=="C"))
    snp[["keep"]][ a1 != "A" & a1 != "T" & a1 != "G" & a1 != "C" ] = F
    snp[["keep"]][ a2 != "A" & a2 != "T" & a2 != "G" & a2 != "C" ] = F
    snp[["flip"]] = (a1 == ref2 & a2 == ref1) | (a1 == flip2 & a2 == flip1)

    return(snp)
}

 allele.process = function(sumstat,name,n,stat){
    qc = allele.qc(sumstat$V2.y,sumstat$V3.y,sumstat$V2.x,sumstat$V3.x)
    # Flip Z-scores for mismatching alleles
    qc$flip = qc$flip%>%which%>%na.omit
    qc$keep = qc$keep%>%which%>%na.omit
    stat = which(colnames(sumstat) %in% stat)
    sumstat[ qc$flip,stat ] = -1 * sumstat[ qc$flip,stat ]
    sumstat$V2.y[ qc$flip ] = sumstat$V2.x[ qc$flip ]
    sumstat$V3.y[ qc$flip ] = sumstat$V3.x[ qc$flip ]
    # Remove strand ambiguous SNPs
        sumstat = sumstat[qc$keep,]
    # Merge the tags
        sumstat = sumstat%>%mutate(rowname.y = pmap_chr( list(V1,V2.y,V3.y),function(a,b,c) paste(c(a,b,c),collapse = "_")),
                                    rowname.x = coalesce(rowname.x,NA,rowname.y),
                                    V2.x = coalesce(V2.x,NA,V2.y),
                                    V3.x = coalesce(V3.x,NA,V3.y),
                                    )%>%select(-rowname.y,-V2.y,-V3.y)
    # Clean name
    colnames(sumstat)[c(1,2,4,5,n+4)] = c("rowname","bhat","V2","V3",name[n])
    return(sumstat)
    }


  ## Start


Theme = read_delim('/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis/Theme_list.txt', delim = "\t")
Theme = Theme%>%mutate(dir = map_chr(`#Theme`,~paste(c(`.x`,"/ENSG00000015475.rds"),collapse = "")),
                                      i = 1:nrow(Theme),
                                      bhat = map(dir, ~readRDS(.x)$bhat%>%as.data.frame%>%rownames_to_column%>%rename(bhat = 2)%>%mutate(snps = map(rowname,~read.table(text = .x, sep = "_",colClass = "character")%>%as_tibble ))%>%unnest(snps)),
                                      sbhat = map(dir, ~readRDS(.x)$sbhat%>%as.data.frame%>%rownames_to_column%>%rename(sbhat = 2)%>%mutate(snps = map(rowname,~read.table(text = .x, sep = "_",colClass = "character")%>%as_tibble ))%>%unnest(snps))
                                      )
n = nrow(Theme)
name = Theme%>%mutate(name = map(`#Theme`, ~read.table(text = .x,sep = "/")),
                            name = map_chr(name, ~.x[,ncol(.x)-2]%>%as.character) )%>%pull(name)


# Join first two conditions
genos_join_bhat = full_join((Theme%>%pull(bhat))[[1]],(Theme%>%pull(bhat))[[2]],by = "V1")

genos_join_bhat = allele.process(genos_join_bhat,name,2,"bhat.y")



genos_join_sbhat = full_join((Theme%>%pull(sbhat))[[1]],(Theme%>%pull(sbhat))[[2]],by = "V1")

genos_join_sbhat  = allele.process(genos_join_sbhat,name,2,"sbhat.y")


# If there are more conditions, join the rest
if(n > 2){
    for(j in 3:n){
        genos_join_bhat = full_join(genos_join_bhat,(Theme%>%pull(bhat))[[j]],by = "V1")
        genos_join_bhat = allele.process(genos_join_bhat,name,j,"bhat.y")
        genos_join_sbhat = full_join(genos_join_sbhat,(Theme%>%pull(sbhat))[[j]],by = "V1")
        genos_join_sbhat  = allele.process(genos_join_sbhat,name,j,"sbhat.y")
    }
}
colnames(genos_join_bhat)[2] = name[1]
colnames(genos_join_sbhat)[2] = name[1]

snps = genos_join_bhat$rowname
genos_join_bhat = genos_join_bhat%>%select(-rowname,-V1,-V2,-V3)%>%as.matrix
genos_join_sbhat = genos_join_sbhat%>%select(-rowname,-V1,-V2,-V3)%>%as.matrix%>%abs


── Column specification ────────────────────────────────────────────────────────
cols(
  `#Theme` = col_character()
)




ERROR: Error: Names repair functions can't return `NA` values.


In [17]:
library("dplyr")
library("tibble")
library("purrr")
library("readr")
library("tidyr")

## Define function
allele.qc  = function(a1,a2,ref1,ref2) {
        a1 = toupper(a1)
        a2 = toupper(a2)
        ref1 = toupper(ref1)
        ref2 = toupper(ref2)

    ref = ref1
    flip = ref
    flip[ref == "A"] = "T"
    flip[ref == "T"] = "A"
    flip[ref == "G"] = "C"
    flip[ref == "C"] = "G"
    flip1 = flip

    ref = ref2
    flip = ref
    flip[ref == "A"] = "T"
    flip[ref == "T"] = "A"
    flip[ref == "G"] = "C"
    flip[ref == "C"] = "G"
    flip2 = flip;

    snp = list()
    snp[["keep"]] = !((a1=="A" & a2=="T") | (a1=="T" & a2=="A") | (a1=="C" & a2=="G") | (a1=="G" & a2=="C"))
    snp[["keep"]][ a1 != "A" & a1 != "T" & a1 != "G" & a1 != "C" ] = F
    snp[["keep"]][ a2 != "A" & a2 != "T" & a2 != "G" & a2 != "C" ] = F
    snp[["flip"]] = (a1 == ref2 & a2 == ref1) | (a1 == flip2 & a2 == flip1)

    return(snp)
}

 allele.process = function(sumstat,name,n,stat){
    qc = allele.qc(sumstat$V2.y,sumstat$V3.y,sumstat$V2.x,sumstat$V3.x)
    # Flip Z-scores for mismatching alleles
    qc$flip = qc$flip%>%which%>%na.omit
    qc$keep = qc$keep%>%which%>%na.omit
    stat = which(colnames(sumstat) %in% stat)
    sumstat[ qc$flip,stat ] = -1 * sumstat[ qc$flip,stat ]
    sumstat$V2.y[ qc$flip ] = sumstat$V2.x[ qc$flip ]
    sumstat$V3.y[ qc$flip ] = sumstat$V3.x[ qc$flip ]
    # Remove strand ambiguous SNPs
        sumstat = sumstat[qc$keep,]
    # Merge the tags
        sumstat = sumstat%>%mutate(rowname.y = pmap_chr( list(V1,V2.y,V3.y),function(a,b,c) paste(c(a,b,c),collapse = "_")),
                                    rowname.x = coalesce(rowname.x,NA,rowname.y),
                                    V2.x = coalesce(V2.x,NA,V2.y),
                                    V3.x = coalesce(V3.x,NA,V3.y),
                                    )%>%select(-rowname.y,-V2.y,-V3.y)
    # Clean name
    colnames(sumstat)[c(1,2,4,5,n+4)] = c("rowname","bhat","V2","V3",name[n])
    return(sumstat)
    }


  ## Start


Theme = read_delim('/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis/Theme_list.txt', delim = "\t")
Theme = Theme%>%mutate(dir = map_chr(`#Theme`,~paste(c(`.x`,"/Analysis_Unit"),collapse = "")),
                                      i = 1:nrow(Theme),
                                      bhat = map(dir, ~readRDS(.x)$bhat%>%as.data.frame%>%rownames_to_column%>%rename(bhat = 2)%>%mutate(snps = map(rowname,~read.table(text = .x, sep = "_",colClass = "character")%>%as_tibble ))%>%unnest(snps)),
                                      sbhat = map(dir, ~readRDS(.x)$sbhat%>%as.data.frame%>%rownames_to_column%>%rename(sbhat = 2)%>%mutate(snps = map(rowname,~read.table(text = .x, sep = "_",colClass = "character")%>%as_tibble ))%>%unnest(snps))
                                      )



── Column specification ────────────────────────────────────────────────────────
cols(
  `#Theme` = col_character()
)


Warning message in gzfile(file, "rb"):
“cannot open compressed file '/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis//sumstat/PCC/Analysis_Unit', probable reason 'No such file or directory'”


ERROR: Error: Problem with `mutate()` input `bhat`.
[31m✖[39m cannot open the connection
[34mℹ[39m Input `bhat` is `map(...)`.


In [ ]:
n = nrow(Theme)
name = Theme%>%mutate(name = map(`#Theme`, ~read.table(text = .x,sep = "/")),
                            name = map_chr(name, ~.x[,ncol(.x)]%>%as.character) )%>%pull(name)


# Join first two conditions
genos_join_bhat = full_join((Theme%>%pull(bhat))[[1]],(Theme%>%pull(bhat))[[2]],by = "V1")

genos_join_bhat = allele.process(genos_join_bhat,name,2,"bhat.y")



genos_join_sbhat = full_join((Theme%>%pull(sbhat))[[1]],(Theme%>%pull(sbhat))[[2]],by = "V1")

genos_join_sbhat  = allele.process(genos_join_sbhat,name,2,"sbhat.y")


# If there are more conditions, join the rest
if(n > 2){
    for(j in 3:n){
        genos_join_bhat = full_join(genos_join_bhat,(Theme%>%pull(bhat))[[j]],by = "V1")
        genos_join_bhat = allele.process(genos_join_bhat,name,j,"bhat.y")
        genos_join_sbhat = full_join(genos_join_sbhat,(Theme%>%pull(sbhat))[[j]],by = "V1")
        genos_join_sbhat  = allele.process(genos_join_sbhat,name,j,"sbhat.y")
    }
}
colnames(genos_join_bhat)[2] = name[1]
colnames(genos_join_sbhat)[2] = name[1]

snps = genos_join_bhat$rowname
genos_join_bhat = genos_join_bhat%>%select(-rowname,-V1,-V2,-V3)%>%as.matrix
genos_join_sbhat = genos_join_sbhat%>%select(-rowname,-V1,-V2,-V3)%>%as.matrix%>%abs

# save the rds file
saveRDS(file = "/mnt/mfs/statgen/xqtl_workflow_testing/testing_6/Meta_analysis/sumstat/RDS/Analysis_Unit", list(bhat=genos_join_bhat, sbhat=genos_join_sbhat,Z = genos_join_bhat/genos_join_sbhat ,snps = snps))